# Medicare DME Fraud Analysis (Executive Overview)

## Introduction
In this notebook, we combine **Medicare DME supplier data (2018–2022)** with a **fraud dataset** identifying organizations under investigation. We **calculate** year-over-year growth metrics to spot suspicious billing patterns and assign a **risk score** to highlight potential fraudulent activity.

### Key Fraud Indicators
1. **Extreme Year-Over-Year (YoY) Swings**: Large jumps (or drops) in claims, charges, or payments often signal abnormal billing.
2. **Outlier Ratios**: Very high average charges per claim, or a payment-to-charge ratio that is unusually low or high, can reflect “phantom” or inflated billing.
3. **“One-and-Done” or “Spike-and-Drop”**: Some suppliers appear for 1–2 years with massive billing, then essentially vanish.
4. **High Services per Claim**: An unusually large number of services or beneficiaries can be another red flag.
5. **Known Fraud Topics Match**: If the organization is flagged for “Health Care Fraud,” “Kickbacks,” or “Opioid” abuse, these patterns reinforce the suspicion.

### Putting It All Together
We load the data, merge on `NPI`, compute derived metrics, **avoid KeyErrors** if a supplier only appears in one year, and assign a **simple `risk_score`**. Then we **highlight** real examples from the dataset that exhibit the typical fraud patterns described above.

## 1. Libraries and Configuration

In [35]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Key columns for analyzing DME suppliers
KEY_METRICS = [
    'Tot_Suplr_Benes',             # Total unique beneficiaries (<11 are suppressed)
    'Tot_Suplr_Clms',              # Total DMEPOS claims submitted
    'Tot_Suplr_Srvcs',             # Total DMEPOS products/services rendered
    'Suplr_Sbmtd_Chrgs',           # Total charges submitted for DMEPOS products/services
    'Suplr_Mdcr_Alowd_Amt',        # Total Medicare allowed amount
    'Suplr_Mdcr_Pymt_Amt',         # Amount Medicare paid after deductible/coinsurance
    'Suplr_Mdcr_Stdzd_Pymt_Amt'    # Standardized Medicare payments
]

# Columns we'll keep for final analysis
RELEVANT_COLUMNS = [
    'Suplr_NPI',                    # Supplier NPI - NPI for the Supplier on the DMEPOS claim
    'Suplr_Prvdr_Last_Name_Org',    # Supplier Last Name/Organization Name
    'Suplr_Prvdr_Spclty_Desc',      # Supplier Provider Specialty Description
    'Suplr_Prvdr_State_Abrvtn',     # Supplier State - State postal abbreviation
    'year',                         # Year of the data
    'Tot_Suplr_Benes',              # Total unique beneficiaries (<11 are suppressed)
    'Tot_Suplr_Clms',               # Total DMEPOS claims submitted
    'Tot_Suplr_Srvcs',              # Total DMEPOS products/services rendered
    'Suplr_Sbmtd_Chrgs',            # Total charges submitted for DMEPOS products/services
    'Suplr_Mdcr_Alowd_Amt',         # Total Medicare allowed amount
    'Suplr_Mdcr_Pymt_Amt',          # Amount Medicare paid after deductible/coinsurance
    'Suplr_Mdcr_Stdzd_Pymt_Amt',    # Standardized Medicare payments
    'has_existing_fraud_case',      # Flag indicating if supplier has an existing fraud case
    'topics',                       # Topics associated with the supplier
    'url'                           # URL for additional information
]

## 2. Load Supplier & Fraud Data
We’ll create functions to load CSVs from `data/<year>/*.csv` (for supplier data) and `data/fraud_data/*.csv` (for fraud).

In [36]:
def load_supplier_data(start_year=2018, end_year=2022):
    """Load DME supplier CSVs from data/<year>/*.csv and combine them."""
    dfs = []
    for yr in range(start_year, end_year+1):
        year_dir = Path(f"data/{yr}")
        csv_files = list(year_dir.glob("*.csv"))
        if not csv_files:
            print(f"No CSV files found for {yr}")
            continue
        for csv_file in csv_files:
            print(f"Loading {csv_file}...")
            df_temp = pd.read_csv(csv_file, low_memory=False)
            df_temp['year'] = yr
            dfs.append(df_temp)
    if not dfs:
        raise ValueError("No supplier data loaded.")
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"Combined supplier DataFrame shape: {combined_df.shape}")
    return combined_df

def load_fraud_data(fraud_path="data/fraud_data/fraud-export.csv"):
    """Load a CSV with fraud cases. Must have at least 'npi' and 'has_existing_fraud_case'."""
    print(f"Loading fraud data from {fraud_path}")
    fraud_df = pd.read_csv(fraud_path, low_memory=False)
    print(f"Fraud rows: {len(fraud_df)}")
    # Convert NPI to numeric and drop missing
    fraud_df['npi'] = pd.to_numeric(fraud_df['npi'], errors='coerce')
    fraud_df = fraud_df.dropna(subset=['npi'])
    fraud_df['npi'] = fraud_df['npi'].astype('int64')

    # Convert textual 'true'/'false' to boolean if needed
    if 'has_existing_fraud_case' in fraud_df.columns and fraud_df['has_existing_fraud_case'].dtype == 'object':
        fraud_df['has_existing_fraud_case'] = fraud_df['has_existing_fraud_case'].map({'true': True, 'false': False})
    return fraud_df

def merge_fraud_suppliers():
    """Load both data sets, keep confirmed fraud, merge on NPI, and filter to relevant columns."""
    supplier_df = load_supplier_data()
    fraud_df = load_fraud_data()

    # rename npi->Suplr_NPI
    fraud_df = fraud_df.rename(columns={'npi':'Suplr_NPI'})
    # only keep suppliers with has_existing_fraud_case = True
    confirmed_fraud = fraud_df[fraud_df['has_existing_fraud_case'] == True]
    confirmed_fraud = confirmed_fraud.drop_duplicates(subset=['Suplr_NPI'], keep='first')
    print(f"Unique NPIs with confirmed fraud: {len(confirmed_fraud)}")

    merged = supplier_df.merge(confirmed_fraud, on='Suplr_NPI', how='inner')
    print(f"Merged shape: {merged.shape}")

    available_cols = [c for c in RELEVANT_COLUMNS if c in merged.columns]
    final_df = merged[available_cols]
    print(f"Final fraud suppliers shape: {final_df.shape}")
    print(f"Unique NPIs: {final_df['Suplr_NPI'].nunique()}")
    return final_df

## 3. Calculate Derived Metrics & Year-over-Year Growth
We’ll add average charge per claim, average payment per claim, etc., then compute YOY changes in claims, charges, payments, and beneficiaries. We must be careful that suppliers appearing only once in given year and hence we can't compare. 

In [37]:
def calculate_derived_metrics(df):
    dfc = df.copy()
    dfc['avg_charge_per_claim'] = dfc['Suplr_Sbmtd_Chrgs'] / dfc['Tot_Suplr_Clms'].replace(0, np.nan)
    dfc['avg_payment_per_claim'] = dfc['Suplr_Mdcr_Pymt_Amt'] / dfc['Tot_Suplr_Clms'].replace(0, np.nan)
    dfc['avg_services_per_claim'] = dfc['Tot_Suplr_Srvcs'] / dfc['Tot_Suplr_Clms'].replace(0, np.nan)
    dfc['payment_to_charge_ratio'] = dfc['Suplr_Mdcr_Pymt_Amt'] / dfc['Suplr_Sbmtd_Chrgs'].replace(0, np.nan)
    return dfc

def calculate_yoy_growth(df):
    """Compute YOY growth in claims, charges, payments, bene."""
    dfx = df.copy()
    for npi in dfx['Suplr_NPI'].unique():
        sub = dfx[dfx['Suplr_NPI'] == npi].sort_values('year')
        if len(sub) > 1:
            for i in range(1, len(sub)):
                prev_idx = sub.index[i-1]
                curr_idx = sub.index[i]
                combos = {
                    'Tot_Suplr_Clms': 'yoy_growth_claims',
                    'Suplr_Sbmtd_Chrgs': 'yoy_growth_charges',
                    'Suplr_Mdcr_Pymt_Amt': 'yoy_growth_payments',
                    'Tot_Suplr_Benes': 'yoy_growth_beneficiaries'
                }
                for metric_col, yoy_col in combos.items():
                    prev_val = sub.loc[prev_idx, metric_col]
                    curr_val = sub.loc[curr_idx, metric_col]
                    if prev_val > 0:
                        yoy = (curr_val - prev_val)/prev_val * 100
                    else:
                        yoy = np.nan
                    dfx.loc[curr_idx, yoy_col] = yoy
    return dfx

def create_time_series_dataset(df):
    df_sorted = df.sort_values(['Suplr_NPI','year'])
    with_metrics = calculate_derived_metrics(df_sorted)
    yoy_final = calculate_yoy_growth(with_metrics)
    return yoy_final

## 4. Assign a Simple Fraud Risk Score - Future EliotNest score :) 
We define a **risk_score** column, incremented by 1 for each suspicious pattern:
1. YOY growth in claims/charges/payments over 200%
2. `avg_charge_per_claim` > $10k
3. `payment_to_charge_ratio` < 0.1 or > 0.9

In [38]:
def assign_fraud_risk_score(df):
    dfx = df.copy()
    dfx['risk_score'] = 0

    yoy_claims = dfx.get('yoy_growth_claims', pd.Series([np.nan]*len(dfx))).fillna(0)
    yoy_charges = dfx.get('yoy_growth_charges', pd.Series([np.nan]*len(dfx))).fillna(0)
    yoy_payments = dfx.get('yoy_growth_payments', pd.Series([np.nan]*len(dfx))).fillna(0)

    # YOY > 200% => +1
    dfx.loc[yoy_claims > 200, 'risk_score'] += 1
    dfx.loc[yoy_charges > 200, 'risk_score'] += 1
    dfx.loc[yoy_payments > 200, 'risk_score'] += 1

    # High average charge (> 10k)
    avg_charge = dfx.get('avg_charge_per_claim', pd.Series([0]*len(dfx))).fillna(0)
    dfx.loc[avg_charge > 10000, 'risk_score'] += 1

    # Payment-to-charge ratio outliers
    pay_ratio = dfx.get('payment_to_charge_ratio', pd.Series([0]*len(dfx))).fillna(0)
    ratio_mask = (pay_ratio < 0.1) | (pay_ratio > 0.9)
    dfx.loc[ratio_mask, 'risk_score'] += 1

    return dfx

## 5. Main Execution Flow
We:
1. **Merge** the data from both sources.
2. **Create** the time-series dataset with YOY columns.
3. **Assign** the risk score.
4. **Display** top suspicious records.
5. **Save** final data to CSV.

### Fraud Patterns Explanation
Below is the extended executive-friendly explanation of typical **red flags** we’re seeing in the data.

1) **Extreme Year-Over-Year (YoY) Swings**
   - Example: **All American Medical Supplies, LLC (NPI 1073935862)** jumps from 688 claims to 46,343 (a ~6300% spike), then crashes by -92%. This is typical of a "spike then crash" pattern that emerges after a crackdown.
   - Another example: **Acqualina Health Medical Solutions (NPI 1265033591)** jumps from 19 claims (2021) to 202 (2022), plus charges from $41k to $1.15M—over 2700%.
   - Such wild changes often indicate questionable billing ramp-up or a rapid shutdown.

2) **Outlier Ratios** (Payment-to-Charge, Average Charges per Claim)
   - Some indicted suppliers show very high average charges (thousands or tens of thousands per claim), or a ratio near 0 or near 1.
   - **Liberty Medical DME (NPI 1134608292)** sees charges of ~$1.58M, then soared, then crashed.
   - **Acqualina** again leaps from ~$41k to $1.15M, with a ~0.77 ratio.
   - A ratio far above or below normal suggests inflated or phantom billing.

3) **One-and-Done or Spike-and-Drop Patterns**
   - Some suppliers exist for 1–2 years with big billings, then vanish.
   - **Central DME Inc (NPI 1326572405)**: from >3,000 claims / $5.7M in charges to 36 claims / $53k next year (a 99% drop).
   - **Pipeline Medical (NPI 1457863326)**: from 1,752 beneficiaries / $2.9M to only 200 beneficiaries / $400k in the next year.
   - This “boom then bust” is often seen in short-lived fraud setups.

4) **High Services per Claim / High Beneficiaries**
   - Example: **Alpine Medical (NPI 1467974428)**: expansions in services per claim or big average charges.
   - **K -Va -T Food Stores Inc (NPI 1083631568)**: ~18,077 services on just 80 claims in 2020 (225+ services/claim). Possibly suspicious.

5) **Known Fraud Topics Align**
   - Many flagged suppliers have topics such as "Health Care Fraud," "Kickbacks," "Opioids," or "False Claims." The patterns support these allegations.

### Summarizing the Patterns
- **Huge Jumps/Crashes** year over year.
- **Outlier Ratios** (payment-to-charge, average charges).
- **Short-Lived Booms** and subsequent bust.
- **Matches** official fraud topics in the second dataset.

All these factors—abnormally rapid growth, big swings, outlier ratios, and alignment with known charges—are classic red flags.

In [39]:
def main():
    # 1) Merge data
    fraud_suppliers_df = merge_fraud_suppliers()

    # 2) Create time series dataset
    time_series_df = create_time_series_dataset(fraud_suppliers_df)

    # 3) Assign risk score
    scored_df = assign_fraud_risk_score(time_series_df)

    # Show top suspicious by risk score
    top_susp = scored_df.sort_values('risk_score', ascending=False)
    print("\n===== Top 10 by Fraud Risk Score =====")
    display(top_susp.head(10)[[
        'Suplr_NPI','Suplr_Prvdr_Last_Name_Org','year','risk_score',
        'yoy_growth_claims','yoy_growth_charges','yoy_growth_payments','avg_charge_per_claim','payment_to_charge_ratio'
    ]])

    # Save the final results
    time_series_df.to_csv("fraud_time_series_fixed.csv", index=False)
    scored_df.to_csv("fraud_scored_fixed.csv", index=False)
    print("Saved final CSVs: 'fraud_time_series_fixed.csv' and 'fraud_scored_fixed.csv'.")

    print("\n=== Conclusion ===")
    print("- We identified multiple suppliers with extremely high YOY changes, outlier ratios, and short-lived spikes.")
    print("- These patterns align closely with known fraud topics (e.g., health care fraud, false claims).")
    print("- The 'risk_score' approach flagged those same suppliers, confirming the typical red flags.")
    print("- Executives can focus on these high-risk suppliers for further investigation.")

## 6. Run the Notebook
Click **Cell → Run All** or **Shift+Enter** on each cell to:
1. Load data.
2. Merge fraud cases.
3. Calculate YOY growth.
4. Assign risk scores.
5. Print top suspicious examples.
6. Save final CSVs.

In [40]:
if __name__ == "__main__":
    main()

Loading data/2018/mup_dme_ry24_p05_v10_dy18_supr.csv...
Loading data/2019/mup_dme_ry24_p05_v10_dy19_supr.csv...
Loading data/2020/mup_dme_ry24_p05_v10_dy20_supr.csv...
Loading data/2021/mup_dme_ry24_p05_v10_dy21_supr.csv...
Loading data/2022/mup_dme_ry24_p05_v10_dy22_supr.csv...
Combined supplier DataFrame shape: (352611, 95)
Loading fraud data from data/fraud_data/fraud-export.csv
Fraud rows: 2085
Unique NPIs with confirmed fraud: 519
Merged shape: (108, 104)
Final fraud suppliers shape: (108, 15)
Unique NPIs: 44

===== Top 10 by Fraud Risk Score =====


,Suplr_NPI,Suplr_Prvdr_Last_Name_Org,year,risk_score,yoy_growth_claims,yoy_growth_charges,yoy_growth_payments,avg_charge_per_claim,payment_to_charge_ratio
61,1134608292,"Liberty Medical Dme, Llc",2020,3,1448.148148,1779.474402,2007.692669,1880.357261,0.770239
73,1861930166,Key Medical Llc,2020,3,336.636245,239.987594,211.664602,1011.392231,0.630318
29,1073935862,"All American Medical Supplies, Llc",2019,3,10766.569767,6064.881735,1086.667497,267.277086,0.128765
44,1578052627,"Absolute Comfort Medical, Inc.",2019,3,398.773006,378.553831,366.096323,1193.507279,0.647649
72,1841725314,Medihealth Medical Solutions Llc,2020,3,308.128079,501.638939,541.235984,1940.289209,0.776156
47,1841725314,Medihealth Medical Solutions Llc,2019,3,600.000000,545.772705,516.409983,1316.215517,0.728227
71,1831652734,Limitless Medical Supplies Llc,2020,3,1213.043478,1274.224634,1306.680871,2054.056291,0.637979
41,1457863326,Pipeline Medical Equipment Inc,2019,3,416.363636,393.237206,411.301333,1032.512912,0.555445
37,1316423015,"Ortho-Med Solution, Inc",2019,3,382.702703,556.187996,577.312412,8734.163494,0.713930
97,1265033591,"Acqualina Health Medical Solutions, Inc.",2022,3,963.157895,2704.320142,4332.811494,5692.714356,0.776182


Saved final CSVs: 'fraud_time_series_fixed.csv' and 'fraud_scored_fixed.csv'.

=== Conclusion ===
- We identified multiple suppliers with extremely high YOY changes, outlier ratios, and short-lived spikes.
- These patterns align closely with known fraud topics (e.g., health care fraud, false claims).
- The 'risk_score' approach flagged those same suppliers, confirming the typical red flags.
- Executives can focus on these high-risk suppliers for further investigation.
